Import libraires

In [41]:
import torch
import torch.nn as nn
import tiktoken

Confriguration_used

In [42]:
GPT_CONFIG_124M = {
    "vocab_size": 50527,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers" :12,
    "drop_rate": 0.1,
    "qkv_bias": False
}

Part1: Dummy GPT class 

In [43]:
class DummyGPTModel(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"],cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self,in_idx):
            batch_size,seq_len = in_idx.shape
            tok_embeds = self.tok_emb(in_idx)
            pos_embeds = self.pos_emb(torch.arange(seq_len,device=in_idx.device))
            x = tok_embeds + pos_embeds
            x= self.drop_emb(x)
            x = self.trf_blocks(x)
            x = self.final_norm(x)
            logits = self.out_head(x)
            return logits
        
class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # A simple placeholder

    def forward(self, x):
        # This block does nothing and just returns its input.
        return x
    
class DummyLayerNorm(nn.Module):
    def __init__(slef,normalized_shape,eps=1e-5):
        super().__init__()

    def forward(slef,x):
        return x

Step1: Tokenization

In [44]:
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2= "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


Step2: Create an instance of dummy gpt model

In [45]:
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)
logits = model(batch)
print("Output shape:", logits.shape)
print(logits)

Output shape: torch.Size([2, 4, 50527])
tensor([[[-0.4811,  0.4325, -0.6385,  ..., -0.9398, -0.1178,  0.1945],
         [-0.0843,  0.4519,  0.2060,  ..., -0.5766, -0.5318, -1.4576],
         [ 0.7199,  0.0575,  0.2597,  ..., -1.0028, -1.9158, -0.5157],
         [ 1.8422,  0.0252,  0.9573,  ..., -0.0315, -0.2424, -0.2487]],

        [[-0.9559,  1.3580, -0.4468,  ..., -0.5588,  0.4615, -0.3940],
         [-0.5088,  0.2881,  0.5815,  ..., -0.0353,  0.1133, -0.5654],
         [ 0.0683, -1.4644,  1.1000,  ..., -1.2006, -0.6936, -0.2516],
         [ 0.6819, -0.8203,  1.2607,  ...,  1.1441, -0.7271, -2.8555]]],
       grad_fn=<UnsafeViewBackward0>)
